In [1]:
import open3d.core as o3c
import numpy as np

# Tensor

Tensor is a "view" of a data Blob with shape, stride, and a data pointer. It is a multidimensional and homogeneous matrix containing elements of single data type. It is used in Open3D to perform numerical operations. It supports GPU operations as well.

## Tensor creation

Tensor can be created from list, numpy array, another tensor. A tensor of specific data type and device can be constructed by passing a ```o3c.Dtype``` and/or ```o3c.Device``` to a constructor. If not passed, the default data type is inferred from the data, and the default device is CPU.
Note that while creating tensor from a list or numpy array, the underlying memory is not shared and a copy is created.

In [ ]:
# Tensor from list.
a = o3c.Tensor([0, 1, 2])
print("Created from list:\n{}".format(a))

# Tensor from Numpy.
a = o3c.Tensor(np.array([0, 1, 2]))
print("\nCreated from numpy array:\n{}".format(a))

# Dtype and inferred from list.
a_float = o3c.Tensor([0.0, 1.0, 2.0])
print("\nDefault dtype and device:\n{}".format(a_float))

# Specify dtype.
a = o3c.Tensor(np.array([0, 1, 2]), dtype=o3c.Dtype.Float64)
print("\nSpecified data type:\n{}".format(a))

# Specify device.
a = o3c.Tensor(np.array([0, 1, 2]), device=o3c.Device("CUDA:0"))
print("\nSpecified device:\n{}".format(a))

   Tensor can also be created from another tensor by invoking the copy constructor. This is a shallow copy, the data_ptr will be copied but the memory it points to will not be copied.

In [ ]:
# Shallow copy constructor.
vals = np.array([1, 2, 3])
src = o3c.Tensor(vals)
dst = src
src[0] += 10

# Changes in one will get reflected in other.
print("Source tensor:\n{}".format(src))
print("\nTarget tensor:\n{}".format(dst))


## Properties of a tensor

In [ ]:
vals = np.array((range(24))).reshape(2, 3, 4)
a = o3c.Tensor(vals,
               dtype=o3c.Dtype.Float64,
               device=o3c.Device("CUDA:0"))
print(f"a.shape: {a.shape}")
print(f"a.strides: {a.strides}")
print(f"a.dtype: {a.dtype}")
print(f"a.device: {a.device}")
print(f"a.ndim: {a.ndim}")

## Copy & device transfer
We can transfer tensors across host and multiple devices.

In [ ]:
# Host -> Device.
a_cpu = o3c.Tensor([0, 1, 2])
a_gpu = a_cpu.cuda(0)
print(a_gpu)

# Device -> Host.
a_gpu = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_cpu = a_gpu.cpu()
print(a_cpu)

# Device -> another Device.
a_gpu_0 = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_gpu_1 = a_gpu_0.cuda(1)
print(a_gpu_1)

## Data Types

Open3d defines seven tensor data types.

| Data type                | dtype               |
|--------------------------|---------------------|
| Uninitialized Tensor     | o3c.Dtype.Undefined |
| 32-bit floating point    | o3c.Dtype.Float32   |
| 64-bit floating point    | o3c.Dtype.Float64   |
| 32-bit integer (signed)  | o3c.Dtype.Int32     |
| 64-bit integer (signed)  | o3c.Dtype.Int64     |
| 8-bit integer (unsigned) | o3c.Dtype.UInt8     |
| Boolean                  | o3c.Dtype.Bool      |

### Type casting

In [ ]:
# E.g. float -> int
a = o3c.Tensor([0.1, 1.5, 2.7])
b = a.to(o3c.Dtype.Int32)
print(a)
print(b)

In [ ]:
# E.g. int -> float
a = o3c.Tensor([1, 2, 3])
b = a.to(o3c.Dtype.Float32)
print(a)
print(b)

## Numpy I/O with direct memory map

Tensors created by passing numpy array to the constructor(```o3c.Tensor(np.array(...)```) do not share memory with the numpy aray. To have shared memory, you can use ```o3c.Tensor.from_numpy(...)``` and ```o3c.Tensor.numpy(...)```. Changes in either of them will get reflected in other.

In [ ]:
# Using constructor
np_a = np.ones((5,), dtype=np.int32)
o3_a = o3c.Tensor(np_a)
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to numpy array will not reflect as memory is not shared.
np_a[0] += 100
o3_a[1] += 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

In [ ]:
# From numpy.
np_a = np.ones((5,), dtype=np.int32)
o3_a = o3c.Tensor.from_numpy(np_a)

# Changes to numpy array reflects on open3d Tensor and vice versa.
np_a[0] += 100
o3_a[1] += 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

In [ ]:
# To numpy.
o3_a = o3c.Tensor([1, 1, 1, 1, 1], dtype=o3c.Dtype.Int32)
np_a = o3_a.numpy()

# Changes to numpy array reflects on open3d Tensor and vice versa.
np_a[0] += 100
o3_a[1] += 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

# For CUDA Tensor, call cpu() before calling numpy().
o3_a = o3c.Tensor([1, 1, 1, 1, 1], device=o3c.Device("CUDA:0"))
print(f"\no3_a.cpu().numpy(): {o3_a.cpu().numpy()}")

## PyTorch I/O with DLPack memory map

In [ ]:
import torch

# From PyTorch
th_a = torch.ones((5,)).cuda(0)
o3_a = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

In [ ]:
# To PyTorch
o3_a = o3d.core.Tensor([1, 1, 1, 1, 1], device=o3d.core.Device("CUDA:0"))
th_a = torch.utils.dlpack.from_dlpack(o3_a.to_dlpack())
o3_a = o3d.core.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

## Binary element-wise operation:

Supported element-wise binary operations are:
1. Add (+)
2. Sub (-)
3. Mul (*)
4. Div (/)
5. Add_ (+=)
6. Sub_ (-=)
7. Mul_ (*=)
8. Div_ (/=)

Note that the operands have to be of same Device, dtype and Broadcast compatible.

In [ ]:
a = o3c.Tensor([1, 1, 1], dtype=o3c.Dtype.Float32)
b = o3c.Tensor([2, 2, 2], dtype=o3c.Dtype.Float32)
print("a + b = {}".format(a + b))
print("a - b = {}".format(a - b))
print("a * b = {}".format(a * b))
print("a / b = {}".format(a / b))

Broadcasting follows the same numpy broadcasting rule as given [here](https://numpy.org/doc/stable/user/basics.broadcasting.html).

In [ ]:
# Automated broadcasting.
a = o3c.Tensor(np.ones((2, 3)), dtype=o3c.Dtype.Float32)
b = o3c.Tensor(np.ones((3,)), dtype=o3c.Dtype.Float32)
print("a + b = {}".format(a + b))

## Unary element-wise operation:
Supported unary element-wise operations are:
1. sqrt, sqrt_(inplace))
2. sin, sin_
3. cos, cos_
4. neg, neg_
5. exp, exp_
6. abs, abs_


In [5]:
a = o3c.Tensor([4, 9, 16], dtype=o3c.Dtype.Float32)
print("a = {}\n".format(a))
print("a.sqrt = {}\n".format(a.sqrt()))
print("a.sin = {}\n".format(a.sin()))
print("a.cos = {}\n".format(a.cos()))

# Inplace operation
a.sqrt_()
print(a)

a = [4.0 9.0 16.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fed1572c5e0]

a.sqrt = [2.0 3.0 4.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fed1718fde0]

a.sin = [-0.756802 0.412118 -0.287903]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fed1718fde0]

a.cos = [-0.653644 -0.91113 -0.957659]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fed1718fde0]

[2.0 3.0 4.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fed1572c5e0]


## Reduction:

Open3D supports following reduction operations.
1. sum
2. mean
3. prod
4. min
5. max
6. argmin
7. argmax

In [36]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)
print("a.sum = {}\n".format(a.sum()))
print("a.min = {}\n".format(a.min()))
print("a.ArgMax = {}\n".format(a.argmax()))

a.sum = 276
Tensor[shape={}, stride={}, Int64, CPU:0, 0x7fed1718d550]

a.min = 0
Tensor[shape={}, stride={}, Int64, CPU:0, 0x7fed1718d550]

a.ArgMax = 23
Tensor[shape={}, stride={}, Int64, CPU:0, 0x7fed1718d550]

a.sum(dims=1) = 
[[12 15 18 21],
 [48 51 54 57]]
Tensor[shape={2, 4}, stride={4, 1}, Int64, CPU:0, 0x7fed1576fa20]

{2, 4}
{2, 1, 4}
[[[12 15 18 21]],
 [[48 51 54 57]]]
Tensor[shape={2, 1, 4}, stride={4, 4, 1}, Int64, CPU:0, 0x7fed1576fa20]


In [44]:
# With specified dimension.
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

print("Along dim=0\n{}".format(a.sum(dim=(0))))
print("Along dim=(0, 2)\n{}\n".format(a.sum(dim=(0, 2))))

# Retention of reduced dimension.
print("Shape without retention : {}".format(a.sum(dim=(0, 2)).shape))
print("Shape with retention : {}".format(a.sum(dim=(0, 2), keepdim=True).shape))

Along dim=0
[[12 14 16 18],
 [20 22 24 26],
 [28 30 32 34]]
Tensor[shape={3, 4}, stride={4, 1}, Int64, CPU:0, 0x7fed171a6460]
Along dim=(0, 2)
[60 92 124]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7fed15711f90]

Shape without retention : {3}
Shape with retention : {1, 3, 1}


## Slicing, indexing, getitem, and setitem

Basic slicing is done by passing an integer, slice object(```start:stop:step```), index array or boolean array. Slicing and indexing produce a view of the tensor. Hence any change in it will also get reflected in the original tensor.

In [63]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)
print("a = \n{}\n".format(a))

# Indexing __getitem__.
print("a[1, 2] = {}\n".format(a[1, 2]))

# Slicing __getitem__.
print("a[1:] = \n{}\n".format(a[1:]))

# slice object.
print("a[:, 0:3:2, :] = \n{}\n".format(a[:, 0:3:2, :]))

# Combined __getitem__
print("a[:-1, 0:3:2, 2] = \n{}\n".format(a[:-1, 0:3:2, 2]))

a = 
[[[0 1 2 3],
  [4 5 6 7],
  [8 9 10 11]],
 [[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7fed15760440]

a[1, 2] = [20 21 22 23]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x7fed157604e0]

a[1:] = 
[[[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={1, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7fed157604a0]

a[:, 0:3:2, :] = 
[[[0 1 2 3],
  [8 9 10 11]],
 [[12 13 14 15],
  [20 21 22 23]]]
Tensor[shape={2, 2, 4}, stride={12, 8, 1}, Int64, CPU:0, 0x7fed15760440]

a[:-1, 0:3:2, 2] = 
[[2 10]]
Tensor[shape={1, 2}, stride={12, 8}, Int64, CPU:0, 0x7fed15760450]



In [64]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

# Changes get reflected.
b = a[:-1, 0:3:2, 2]
b[0] += 100
print("b = {}\n".format(b))
print("a = \n{}".format(a))

b = [[102 110]]
Tensor[shape={1, 2}, stride={12, 8}, Int64, CPU:0, 0x7fed156d0f40]

a = 
[[[0 1 102 3],
  [4 5 6 7],
  [8 9 110 11]],
 [[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7fed156d0f30]


In [59]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

# Example __setitem__
a[:, :, 2] += 100
print(a)

[[[0 1 2 3],
  [4 5 6 7],
  [8 9 10 11]],
 [[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7fed15747e40] 

[[[0 1 1002 3],
  [4 5 1006 7],
  [8 9 1010 11]],
 [[12 13 1014 15],
  [16 17 1018 19],
  [20 21 1022 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7fed15747e40]


## Advanced indexing

Advanced indexing is triggered while passing an index array or~ a boolean array or their combination with integer/slice object. 

In [ ]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

# __getitem__
print(a[:, [1, 2], [1, 2]])
print(a[1, [[1, 2], [2, 1]], 0:4:2])

# __setitem__
a[:, 0:2, [1, 2]] = o3c.Tensor(np.array([[100, 200], [300, 400]]))
print(a)

## TensorList

In [ ]:
vals = np.array(range(24), dtype = np.float32).reshape((2, 3, 4))

# Empty TensorList
a = o3c.TensorList([3, 4])
print(a)

# TensorList with single Tensor
b = o3c.TensorList([3, 4], size = 1)
print(b)

# TensorList from tensor
c = o3c.TensorList.from_tensor(o3c.Tensor(vals))
print(c)

# TensorList from multiple tensors
d = o3c.TensorList.from_tensors([o3c.Tensor(vals[0]), o3c.Tensor(vals[1])])
print(d)

# Concatenate TensorLists
print(b + c)
print(o3c.TensorList.concat(b, c))

# Append a Tensor to TensorList
d.push_back(o3c.Tensor(vals[0]))
print(d)

# Append a TensorList to another TensorList
d.extend(b)
print(d)


## Logical operations

Open3D supports following logical operators:
1. logical_and - returns tensor with element wise logical AND.
2. logical_or  - returns tensor with element wise logical OR.
3. logical_xor - returns tensor with element wise logical XOR.
4. logical_not - returns tensor with element wise logical NOT.
5. all         - returns true if all elements in the tensor are true.
6. any         - returns true if any element in the tensor is true.
7. allclose    - returns true if two tensors are element wise equal within a tolerance.
8. isclose     - returns tensor with element wise ```allclose``` operation.
9. issame      - returns true if and only if two tensors are same(even same underlying memory).


In [80]:
a = o3c.Tensor(np.array([True, False, True, False]))
b = o3c.Tensor(np.array([True, True, False, False]))

print("a AND b = {}".format(a.logical_and(b)))
print("a OR b = {}".format(a.logical_or(b)))
print("a XOR b = {}".format(a.logical_xor(b)))
print("NOT a = {}\n".format(a.logical_not()))

# Only works for boolean tensors.
print("a.any = {}".format(a.any()))
print("a.all = {}\n".format(a.all()))

# If tensor is not boolean, 0 will be treated as False, while non-zero as true.
# The tensor will be filled with 0 or 1 casted to tensor's dtype.
c = o3c.Tensor(np.array([2.0, 0.0, 3.5, 0.0]))
d = o3c.Tensor(np.array([0.0, 3.0, 1.5, 0.0]))
print("c AND d = {}".format(c.logical_and(d)))


a AND b = [True False False False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7fed18a14be0]
a OR b = [True True True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7fed15608b10]
a XOR b = [False True True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7fed1565af90]
NOT a = [False True False True]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7fed15663940]

a.any = True
a.all = False

c AND d = [False False True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7fed156d6c20]


In [90]:
a = o3c.Tensor(np.array([1, 2, 3, 4]), dtype=o3c.Dtype.Float64)
b = o3c.Tensor(np.array([1, 1.99999, 3, 4]))

# Throws exception if the device/dtype is not same.
# Returns false if the shape is not same.
print("allclose : {}".format(a.allclose(b)))

# Throws exception if the device/dtype/shape is not same.
print("isclose : {}".format(a.isclose(b)))

# Returns false if the device/dtype/shape/ is not same.
print("issame : {}".format(a.issame(b)))

allclose : True
isclose : [True True True True]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7fed18a94c50]
issame : False


## Comparision Operations

In [98]:
a = o3c.Tensor([0, 1, -1])
b = o3c.Tensor([0, 0, 0])

print("a > b = {}".format(a > b))
print("a >= b = {}".format(a >= b))
print("a < b = {}".format(a < b))
print("a <= b = {}".format(a <= b))
print("a == b = {}".format(a == b))
print("a != b = {}".format(a != b))

# Throws exception if device/dtype is not shape.
# If shape is not same, then tensors should be broadcast compatible.
print("a > b = {}".format(a > b[0]))

a > b = [False True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7fed1bed28f0]
a >= b = [True True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7fed171c4b20]
a < b = [False False True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7fed171c75d0]
a <= b = [True False True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7fed171b26f0]
a == b = [True False False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7fed1718d550]
a != b = [False True True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7fed171d1a30]
a > b = [False True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7fed171ca5f0]


## Nonzero operations

In [67]:
a = o3c.Tensor([[3, 0, 0], [0, 4, 0], [5, 6, 0]])

print(a.nonzero())
print(a.nonzero(as_tuple = 1))

[[0 1 2 2],
 [0 1 0 1]]
Tensor[shape={2, 4}, stride={4, 1}, Int64, CPU:0, 0x7fed156c8680]
[[0 1 2 2]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x7fed1576f7e0], [0 1 0 1]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x7fed15705e60]]


## Boolean operations

In [ ]:
a = o3c.Tensor([1, -1, -2, -3])

a[a < 0]
print(a[a < 0])

a[a > 0] = 2
print(a)

## Scalar operations

In [ ]:
a = o3c.Tensor.ones((2, 3), dtype = o3c.Dtype.Float32)

print(a.add(1))
print(a + 1)
print(a + True)
print(a - 1)
print(a - True)
print(a * 10)
print(10 / a)

# Inplace
a.add_(1)
print(a)
a.sub_(1)
print(a)

# Shorthand
a += 1
print(a)
a -= 1
print(a)
a *= 10
print(a)
a /= 2
print(a)